# **Regressão Linear Multivariada**


## Introdução

Os algoritmos de otimização são usados por algoritmos de aprendizado de máquina para encontrar um bom conjunto de parâmetros do modelo, dado um conjunto de dados de treinamento. O Gradiente Descendente Estocástico é o algoritmo de otimização mais comum usado em aprendizado de máquina. Neste tutorial vamos descobrir, com o Python, como implementar esse algoritmo em conjunto com um algoritmo de Regressão Linear. 

Antes de vermos as implementações, vamos entender alguns conceitos.

### **Regressão Linear Multivariada**
<br />
É um algoritmo de aprendizado de máquina que é responsável por descobrir um conjunto de coeficientes b0 á bn tal que<br />
<br />
<center><strong>y = b0 + b1 \* x1 + b2 \* x2 + ... + bn \* xn</strong> ,</center>

onde cada atributo x é um input e y é uma previsão.<br />
Nesse tutorial, os coeficientes serão encontrados através do Gradiente Descendente Estocástico.


### Gradiente Descendente Estocástico
Processo de minização de uma função seguindo a inclinação ou gradiente da função. No aprendizado de máquina, podemos
usar uma técnica que avalia e atualiza os coeficientes a cada iteração para minimizar o erro de um modelo ou 
dados de treinamento. O modelo faz uma previsão para um treinamento de instância, o erro é calculado e o modelo é 
atualizado para reduzir o erro na próxima previsão. Essa operação é repetida para um número fixo de iterações.

A cada iteração, o valor de b é atualizado para
<center><strong> b = b - taxa de aprendizado \* erro \* x </strong> ,</center>

onde b é o coeficiente sendo otimizado, taxa de aprendizado é um valor que pode e deve ser fornecido de acordo com
a necessidade, erro é o erro da previsão do modelo sobre os dados de treinamento e x é um valor de entrada.



<br />
Neste tutorial usaremos um conjunto de dados sobre qualidade do vinho branco. Nosso exemplo, no final do tutorial, buscará prever a qualidade do vinho branco. Baixe o conjunto de dados através do link https://github.com/DayanaRochaM/TutorialMachineLearning-/blob/master/winequality-white.csv e guarde-o no seu diretório de trabalho atual sem alterar seu nome (<strong>winequality-white.csv</strong>).

## Passos do tutorial:
 1. Fazer previsões: implementar algoritmo que faz previsões e testá-lo com coeficientes previamente fornecidos.
 2. Estimar coeficientes: implementar algoritmo que estima os coeficientes a serem usados nas previsões (Regressão Linear usando Gradiente Descendente Estocástico).
 3. Estudo de caso do vinho de qualidade: testar algoritmos implementados em um dataset sobre a qualidade do vinho branco.

### 1. Fazer previsões

O primeiro passo é desenvolver uma função que faz previsões. Esta vai ser necessária para a avaliação dos possíveis valores de coeficientes. Abaixo está a função chamada **predict()** que prevê um valor de saída para uma linha a partir de um conjunto de coeficientes. Execute-a pois precisaremos dela futuramente neste tutorial.

In [3]:
def predict(row, coefficients):
  yhat = coefficients[0]
  for i in range(len(row)-1):
    yhat += coefficients[i + 1] * row[i]
  return yhat

O primeiro coeficiente é chamado de b0 ou viés, pois é autônomo e não é multiplicados por um valor de entrada (x) como os demais.

Podemos usar o seguinte conjunto simples de dados como dataset para testar nossa função:
<br />
**x**, **y** <br />
1, 1 <br />
2, 3 <br />
4, 3 <br />
3, 2 <br />
5, 5 <br />
<br />
**coef**<br />
0.4<br />
0.8<br />


Podemos testar nossa função com esses valores da seguinte forma:

In [4]:
#Exemplo predict() com coeficientes
dataset = [[1, 1], [2, 3], [4, 3], [3, 2], [5, 5]]
coef = [0.4, 0.8]

for row in dataset:
    yhat = predict(row, coef)
    print("Expected=%.3f, Predicted=%.3f" % (row[-1], yhat))

Expected=1.000, Predicted=1.200
Expected=3.000, Predicted=2.000
Expected=3.000, Predicted=3.600
Expected=2.000, Predicted=2.800
Expected=5.000, Predicted=4.400


Já tendo executado a função **predict()**, execute a célula acima para ver o resultado. Neste exemplo acima, para cada x há dois coeficientes (b0 = 0.4 e <br /> b1 = 0.8). A equação de previsão que modelamos para esse problema é:

**<center>y = b0 + b1 \* x </center>**

Agora estamos prontos para implementar um algoritmo de Regressão Linear usando Gradiente Descendente Estocástico para encontrar nossos coeficientes.

<br />
<br />


### 2. Estimando coeficientes

Podemos estimar os valores de coeficientes para nossos dados de treino usando Gradiente Descendente Estocástico. Esta operação requer dois parâmetros:

* **Taxa de aprendizado**: usado para limitar a quantidade que cada coeficiente é corrigido sempre que é atualizado.

* **Períodos**: o número de vezes para percorrer os dados de treinamento enquanto atualiza os coeficientes.

Juntamente com os dados de treinamento, esses dois argumentos serão usados na função.

A função executará 3 loops: loop por período, loop por linha em cada período, loop por cada coeficiente que 
o atualiza pra cada linha no período. Em resumo, atualizamos cada coeficiente para cada linha nos dados 
de treinamento, em cada período.

Os coeficientes são atualizados baseados nos erros que o modelo fez.

<br />
O erro é calculado como a diferença entre a previsão feita com os coeficientes e o valor de saída esperado. <br />
<br />
<center>**erro = previsão - saída esperada**</center>

<br />
Há um coeficiente para cada x, e eles são atualizados de forma consistente, por exemplo: <br />
<br />
**<center>b1(t+1) = b1(t) - taxa de erro \* erro(t) \* x1(t)</center>**
<br />
<br />
O primeiro coeficiente da lista é atualizado de forma similar, exceto que a fórmula não possui um x associado.
<br />
<br />
**<center>b0(t+1) = b0(t) - taxa de erro * erro(t)</center>**

Agora podemos juntar todas essas informações.<br />
<br />
<br />
Abaixo está a função **coefficients_sgd()** que calcula os valores dos coeficientes de Regressão Linear para um conjunto de dados de treinamento usando Gradiente Descendente Estocástico. Execute-o para que possamos usá-lo posteriormente.

In [6]:
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            yhat = predict(row, coef)
            error = yhat - row[-1]
            sum_error += error**2
            coef[0] = coef[0] - l_rate * error
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] - l_rate * error * row[i] 
                print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    return coef

No código acima, além das operações previamente mencionadas, calculamos a soma do erro ao quadrado de cada período para que possamos imprimi-lo como mensagem no loop externo. Podemos testar esta função no nosso pequeno conjunto de dados já criado (dataset). Devemos criar, entretanto, a taxa de aprendizado e o número de períodos (l_rate e n_epoch, respectivamente encontrados abaixo). Entenda o código abaixo e veja qual o resultado de sua execução.

In [7]:
l_rate = 0.001
n_epoch = 50
coef = coefficients_sgd(dataset, l_rate, n_epoch)
print(coef)

>epoch=0, lrate=0.001, error=1.000
>epoch=0, lrate=0.001, error=9.982
>epoch=0, lrate=0.001, error=18.791
>epoch=0, lrate=0.001, error=22.541
>epoch=0, lrate=0.001, error=46.236
>epoch=1, lrate=0.001, error=0.878
>epoch=1, lrate=0.001, error=9.204
>epoch=1, lrate=0.001, error=16.819
>epoch=1, lrate=0.001, error=19.985
>epoch=1, lrate=0.001, error=41.305
>epoch=2, lrate=0.001, error=0.771
>epoch=2, lrate=0.001, error=8.501
>epoch=2, lrate=0.001, error=15.070
>epoch=2, lrate=0.001, error=17.732
>epoch=2, lrate=0.001, error=36.930
>epoch=3, lrate=0.001, error=0.676
>epoch=3, lrate=0.001, error=7.865
>epoch=3, lrate=0.001, error=13.520
>epoch=3, lrate=0.001, error=15.746
>epoch=3, lrate=0.001, error=33.047
>epoch=4, lrate=0.001, error=0.593
>epoch=4, lrate=0.001, error=7.290
>epoch=4, lrate=0.001, error=12.146
>epoch=4, lrate=0.001, error=13.998
>epoch=4, lrate=0.001, error=29.601
>epoch=5, lrate=0.001, error=0.519
>epoch=5, lrate=0.001, error=6.769
>epoch=5, lrate=0.001, error=10.929
>epo

Podemos observar que o erro vai decaindo até o final do período para cada linha.
Observe que poderíamos treinar por muito mais tempo (mais períodos) ou aumentar a quantidade que atualizamos os 
coeficientes de cada período (taxa de aprendizado mais alta).
<br />
<br />
Sinta-se à vontade para testar as funções acima alterando os valores de entrada das funções.

### 3. Estudo de caso do vinho de qualidade

Agora vamos testar o nosso modelo de Regressão Linear usando Gradiente Descendente Estocástico em um dataset de qualidade de vinho. O exemplo que utilizaremos assume que uma cópia CSV do dataset já está no seu diretório de trabalho com o nome **winequality-white.csv**.

Para trabalhar com os dados carregamos o dataset, os valores strings são convertidos para numéricos float e cada coluna
tem seus valores normalizados para valores no range de 0 a 1. Esses passos são efetuados com as funções auxiliares 
loas_csv() e str_column_to_float() para carregar e preparar o dataset, e dataset_minmax() e normalize_dataset() 
para normalizá-lo.

Usaremos a validação cruzada de k partes para estimar o desempenho do modelo aprendido em dados não vistos. 
Isso significa que vamos construir e avaliar k modelos e estimar o desempenho como o erro médio do modelo. 
O erro quadrático médio da raiz será usado para avaliar cada modelo. Esses comportamentos são fornecidos nas funções 
cross_validation_split(), rmse_metric() e evaluate_algorithm().


Usaremos as funções **predict()**, **coefficients_sgd()** e **linear_regression_sgd()** para treinar o modelo.
Abaixo encontra-se o exemplo completo, analise-o e execute-o.

In [9]:
#Importar funções necessárias
from random import seed
from random import randrange
from csv import reader
from math import sqrt

#Carregar arquivo CSV
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

#Converter colunas string para float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

#Encontrar valores min e max para cada coluna
def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

#Reescalar colunas do dataset para o range 0-1
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
             row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

#Dividir o dataset em k partes
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

#Calcular a média do erro ao quadrado
def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        prediction_error = predicted[i] - actual[i]
        sum_error += (prediction_error ** 2)
    mean_error = sum_error / float(len(actual))
    return sqrt(mean_error)

#Avaliar um algoritmo usando uma divisão de validação cruzada
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        rmse = rmse_metric(actual, predicted)
        scores.append(rmse)
    return scores

#Fazer previsões com coeficientes
def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row)-1):
        yhat += coefficients[i + 1] * row[i]
    return yhat

#Estimar coeficientes de Regressão Linear usando Gradiente Descendente Estocástico
def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        for row in train:
            yhat = predict(row, coef)
            error = yhat - row[-1]
            coef[0] = coef[0] - l_rate * error
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] - l_rate * error * row[i]
            # print(l_rate, n_epoch, error)
    return coef

#Algoritmo de Regressão Lenar com Gradiente Descendente Estocástico
def linear_regression_sgd(train, test, l_rate, n_epoch):
    predictions = list()
    coef = coefficients_sgd(train, l_rate, n_epoch)
    for row in test:
        yhat = predict(row, coef)
        predictions.append(yhat)
    return(predictions)



#Regressão Linear com dataset de qualidade de vinho
seed(1)

# carregando e preparandos dados
filename = 'winequality-white.csv' 
dataset = load_csv(filename)
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
    
# normalizar
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# testar algoritmo
n_folds = 5
l_rate = 0.01
n_epoch = 50
scores = evaluate_algorithm(dataset, linear_regression_sgd, n_folds, l_rate, n_epoch) 
print('Scores: %s' % scores)
print('Mean RMSE: %.3f' % (sum(scores)/float(len(scores))))

Scores: [0.12248058224159092, 0.13034017509167112, 0.12620370547483578, 0.12897687952843237, 0.12446990678682233]
Mean RMSE: 0.126


O k = 5 foi usado para fazer validação cruzada, dando cada parte 4898/5 = 979.6, ou seja, menos de 1000 registros 
avaliados para cada iteração. Como podes observar, foi escolhida uma taxa e aprendizado igual a 0.01 e 50 períodos de treino. No fim do treino, o programa mostra a média dos scores.